<a href="https://colab.research.google.com/github/PalomiBollapalli/Hackathon/blob/main/hackathon_c_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <time.h>
int vulnerable_function(const char *input) {
  char buffer[10];
  if (strlen(input) >= sizeof(buffer)) {
    return -1;
  }
  strcpy(buffer, input);
  return 0;
}
void simple_fuzzer(int iterations) {
  srand(time(NULL));
  for (int i = 0; i < iterations; ++i) {
    int length = rand() % 50;
    char *input = (char *)malloc(length + 1); // +1 for null terminator

    if (input == NULL) {
      perror("malloc failed");
      exit(EXIT_FAILURE);
    }

    for (int j = 0; j < length; ++j) {
      input[j] = rand() % 256; // Random byte value
    }
    input[length] = '\0'; // Null-terminate the string

    int result = vulnerable_function(input);

    if (result == -1) {
      printf("Crash detected with input: %s\n", input);
      // Save the crashing input to a file for analysis
      FILE *crash_file = fopen("crash.txt", "w");
      if (crash_file != NULL) {
          fprintf(crash_file, "%s", input);
          fclose(crash_file);
      }
      free(input);
      return; // Stop fuzzing after a crash
    }
    free(input);
  }
  printf("Fuzzing completed without crashes.\n");
}

// More advanced fuzzer with mutations
void mutation_fuzzer(int iterations) {
    srand(time(NULL));
    char base_input[20] = "initial_input"; // Start with a known input
    int base_length = strlen(base_input);

    for (int i = 0; i < iterations; ++i) {
        char *input = (char *)malloc(base_length + 20); // Allow for mutations
        if (input == NULL) {
            perror("malloc failed");
            exit(EXIT_FAILURE);
        }
        strcpy(input, base_input);
        int current_length = base_length;

        // Apply random mutations
        for (int j = 0; j < rand() % 5; ++j) { // A few mutations per iteration
            int mutation_type = rand() % 3;
            int mutation_pos = rand() % current_length;

            switch (mutation_type) {
                case 0: // Bit flip
                    input[mutation_pos] ^= (1 << (rand() % 8));
                    break;
                case 1: // Byte overwrite
                    input[mutation_pos] = rand() % 256;
                    break;
                case 2: // Byte insertion
                    if(current_length < sizeof(input)-2){ // avoid buffer overflow in mutation.
                        memmove(&input[mutation_pos+1], &input[mutation_pos], current_length - mutation_pos);
                        input[mutation_pos] = rand() % 256;
                        current_length++;
                        input[current_length] = '\0';
                    }
                    break;
            }
        }

        int result = vulnerable_function(input);

        if (result == -1) {
            printf("Crash detected with input: %s\n", input);
            FILE *crash_file = fopen("crash.txt", "w");
            if (crash_file != NULL) {
                fprintf(crash_file, "%s", input);
                fclose(crash_file);
            }

            free(input);
            return;
        }
        free(input);
    }
    printf("Fuzzing completed without crashes.\n");
}

int main() {
  printf("Running simple fuzzer...\n");
  simple_fuzzer(100000); // Adjust iterations as needed

  printf("\nRunning mutation fuzzer...\n");
  mutation_fuzzer(100000);

  return 0;
}